In [6]:
from pandas import concat, DataFrame, read_csv ,get_dummies
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import scale

In [7]:
train = read_csv('./data/train.csv')
test= read_csv('./data/test.csv')
x_train = train.drop(['Survived', 'PassengerId'], axis=1)
y_train = train.Survived
PassengerId = test.PassengerId.to_numpy()
x_test = test.drop(['PassengerId'], axis=1)
data = concat([x_train, x_test])

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    1309 non-null   int64  
 1   Name      1309 non-null   object 
 2   Sex       1309 non-null   object 
 3   Age       1046 non-null   float64
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   Ticket    1309 non-null   object 
 7   Fare      1308 non-null   float64
 8   Cabin     295 non-null    object 
 9   Embarked  1307 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 112.5+ KB


# handling Name column

In [128]:
def title_adjusting(title):
    if title in ['Mr', 'Miss', 'Mrs', 'Ms']:
        return 'English'
    elif title in ['Sir', 'Lady', 'the Countess']:
        return 'EnglishNoble'
    elif title in ['Major', 'Col', 'Capt']:
        return 'Military'   
    elif title in ['Mlle', 'Mme', 'Don', 'Dona', 'Jonkheer']:
        return 'OtherEuropean'
    else:
        return 'others'

In [129]:
def make_one_hot(data, field):
    temp = get_dummies(data[field], prefix=field)
    data.drop(field, axis=1, inplace=True)
    return concat([data, temp], axis=1)

In [130]:
data.Embarked.fillna('blank', inplace=True)
    
data.Cabin.fillna('blank' , inplace=True)
data.Cabin = data.Cabin.map(lambda a: a[0])
    
data['FamiySize'] = data.SibSp + data.Parch + 1
    
data.Name = data.Name.map(
    lambda name: name.split(',')[1].split('.')[0].strip())
    
for i in ['Embarked', 'Cabin', 'Name']:
    data = make_one_hot(data,i)
    
data['Sex'].replace({'male':1, 'female':0}, inplace=True)

data.drop('Ticket', axis=1, inplace=True)

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Pclass             1309 non-null   int64  
 1   Sex                1309 non-null   int64  
 2   Age                1046 non-null   float64
 3   SibSp              1309 non-null   int64  
 4   Parch              1309 non-null   int64  
 5   Fare               1308 non-null   float64
 6   FamiySize          1309 non-null   int64  
 7   Embarked_C         1309 non-null   uint8  
 8   Embarked_Q         1309 non-null   uint8  
 9   Embarked_S         1309 non-null   uint8  
 10  Embarked_blank     1309 non-null   uint8  
 11  Cabin_A            1309 non-null   uint8  
 12  Cabin_B            1309 non-null   uint8  
 13  Cabin_C            1309 non-null   uint8  
 14  Cabin_D            1309 non-null   uint8  
 15  Cabin_E            1309 non-null   uint8  
 16  Cabin_F            1309 n

In [133]:

data = IterativeImputer().fit_transform(data)

from sklearn.tree import DecisionTreeClassifier
x_train , x_test = data[:len(train)], data[len(train):]
cls_dt= DecisionTreeClassifier()
cls_dt.fit(x_train, y_train)
cls_dt.score(x_train, y_train)

0.9876543209876543

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

data_scaled = scale(data)
x_train , x_test = data_scaled[:len(train)], data_scaled[len(train):]

cls_MLP = MLPClassifier(hidden_layer_sizes=len(x_train[0])*2, max_iter=1000)
cls_MLP.fit(x_train,y_train)
scores = cross_val_score(cls_MLP, x_train, y_train, cv=8)
print("Accuracy: {:0.2f} (+/- {:0.2f})".format(scores.mean(), scores.std() * 2))
print(cls_MLP.score(x_train,y_train))

predictions = cls_MLP.predict(x_test)
df = DataFrame(data=list(zip(PassengerId, predictions)), columns=['PassengerId', 'Survived'])
df.to_csv('submission.csv', index=False)



In [ ]:
from sklearn.tree import DecisionTreeClassifier
cls_dt= DecisionTreeClassifier()
cls_dt.fit(x_tr, y_tr)
cls_dt.score(x_ts, y_ts)